In [1]:

import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-189ca005-d852-4370-b4dd-50a8499acec6',
    'iam_service_endpoint': 'https://iam.ng.bluemix.net/oidc/token',
    'api_key': 'mAo40B3h-s0GPQnsakluqLLmGPlQkbNmHif2xzcvDY6C'
}

configuration_name = 'os_9bed24a8267a4d0ea5db2140346dee9d_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df_data_1 = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('ASA_log.csv', 'capstoneproject-donotdelete-pr-68gfgxkh2rslzx'))
df_data_1.take(5)


Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20190916183034-0001
KERNEL_ID = eaeb136b-920d-47c3-95f9-cbd1f0edd06d


[Row(sourcePort='52217', destinationPort='2000', protocolName='tcp_ip', IPgeo=None, deviceId='31410', event_category='5010', categoryDescription='Misc Exploit', eventDescription='Unrecognized Palo Alto PA Series Vulnerability Exploit Threat Event', relevance='8', credibility='10', severity='9', magnitude='9', Event_DateTime='Mar 14, 2019, 8:26:53 PM', eventCount='1'),
 Row(sourcePort='51405', destinationPort='80', protocolName='tcp_ip', IPgeo=None, deviceId='31410', event_category='7024', categoryDescription='Information Leak', eventDescription='A directory traversal vulnerability has been discovered in parsing malformed HTTP requests. This vulnerability is due to a lack of proper checks in HTTP URI requests. A successful attack could result in access to sensitive information that could further aid in other attacks.', relevance='8', credibility='10', severity='1', magnitude='6', Event_DateTime='Mar 14, 2019, 8:26:53 PM', eventCount='1'),
 Row(sourcePort='36002', destinationPort='445', 

In [66]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler, Normalizer
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.sql.functions import unix_timestamp, from_unixtime, to_timestamp
from sklearn.model_selection import train_test_split

df_data_1.createOrReplaceTempView("firewall")
data = spark.sql('SELECT * FROM firewall')

#for col, t in data.dtypes:
#    if t == 'string':
#        try:
#            data = data.withColumn(col, data[col].cast(IntegerType()))
#        except:
#            pass
#data = data.drop('Event_DateTime')        
 
data = data.withColumn('sourcePort', data['sourcePort'].cast(IntegerType()))
data = data.withColumn('destinationPort', data['destinationPort'].cast(IntegerType()))
data = data.withColumn('deviceId', data['deviceId'].cast(IntegerType()))
data = data.withColumn('event_category', data['event_category'].cast(IntegerType()))
data = data.withColumn('relevance', data['relevance'].cast(IntegerType()))
data = data.withColumn('credibility', data['credibility'].cast(IntegerType()))
data = data.withColumn('severity', data['severity'].cast(IntegerType()))
data = data.withColumn('magnitude', data['magnitude'].cast(IntegerType()))
data = data.withColumn('eventCount', data['eventCount'].cast(IntegerType()))
data = data.drop('Event_DateTime')
data = data.fillna('Unknown')
#x = data.drop('magnitude')
#y = data[['magnitude']]
#xtrain, xtest = x.randomSplit([0.66, 0.34])
#ytrain, ytest = y.randomSplit([0.66, 0.34])

#data.printSchema()
#print(data.columns)


In [68]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, Normalizer
#"{0}_indexed".format(c)
#x_train.show()
categorical_columns = ['protocolName', 'IPgeo', 'event_category','eventDescription', 'categoryDescription']
output_columns = ['pnlbl', 'ipglbl', 'evcatlbl', 'ecdlbl', 'catdeslbl']
indexers = [
    StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c), handleInvalid="keep")
    for c in categorical_columns
]

encoders = [OneHotEncoder(dropLast=False, inputCol=indexer.getOutputCol(), outputCol="{0}_encoded".format(indexer.getOutputCol()))
           for indexer in indexers
          ]
assembler = VectorAssembler(inputCols=[encoder.getOutputCol() for encoder in encoders], outputCol="features")
pipeline = Pipeline(stages=indexers + encoders+[assembler])
model=pipeline.fit(data)
transformed = model.transform(data)
dataindex = transformed
for col in categorical_columns:
    dataindex = dataindex.drop(col)
x = dataindex.drop('features')
y = dataindex[['features']]
x.columns
#x.show()
#y.show()
#transformed.show()


['sourcePort',
 'destinationPort',
 'deviceId',
 'relevance',
 'credibility',
 'severity',
 'magnitude',
 'eventCount',
 'protocolName_indexed',
 'IPgeo_indexed',
 'event_category_indexed',
 'eventDescription_indexed',
 'categoryDescription_indexed',
 'protocolName_indexed_encoded',
 'IPgeo_indexed_encoded',
 'event_category_indexed_encoded',
 'eventDescription_indexed_encoded',
 'categoryDescription_indexed_encoded']

In [69]:
column_names = ['sourcePort',
 'destinationPort',
 'deviceId',
 'relevance',
 'credibility',
 'severity',
 'magnitude',
 'eventCount',
 'protocolName_indexed',
 'IPgeo_indexed',
 'event_category_indexed',
 'eventDescription_indexed',
 'categoryDescription_indexed',
 'protocolName_indexed_encoded',
 'IPgeo_indexed_encoded',
 'event_category_indexed_encoded',
 'eventDescription_indexed_encoded',
 'categoryDescription_indexed_encoded']

In [65]:
#datapipe.toPandas()
#datapipe1 = datapipe.toPandas()
#print(datapipe1)
import numpy as np

x_np = np.array(x.select('sourcePort',
 'destinationPort',
 'deviceId',
 'relevance',
 'credibility',
 'severity',
 'magnitude',
 'eventCount',
 'protocolName_indexed',
 'IPgeo_indexed',
 'event_category_indexed',
 'protocolName_indexed_encoded',
 'IPgeo_indexed_encoded',
 'event_category_indexed_encoded',).collect())
#xtestnp = np.array(x.select('sourcePort',
# 'destinationPort',
# 'deviceId',
# 'relevance',
# 'credibility',
# 'severity',
# 'magnitude',
# 'eventCount',
# 'protocolName_indexed',
# 'IPgeo_indexed',
# 'event_category_indexed',
# 'categoryDescription_indexed',
# 'eventDescription_indexed',
# 'protocolName_indexed_encoded',
# 'IPgeo_indexed_encoded',
# 'event_category_indexed_encoded',
# 'categoryDescription_indexed_encoded',
# 'eventDescription_indexed_encoded').collect())
y_np = np.array(y.select('features').collect())
#ytestnp = np.array(y1test.select('magnitude_index').collect())
#print(xtrainnp.shape)
#print(xtestnp.shape)
#print(ytrainnp.shape)
#print(ytestnp.shape)


ValueError: setting an array element with a sequence.

In [79]:
# general summary statistcs are meaningless on this data. box plots of counts for some of the ports might be but graphically this data
# has very little significance
#for col in xtrain.columns:
#    xtrain.describe(col).show()
#
          

In [111]:
#cat_features = data.columns
#stages = []
#for feature in data.columns:
#    indexers = StringIndexer(inputCol=features, outputCol=features+'_index').fit(data)
#    encoder = OneHotEncoderEstimator(inputCol=[indexers.getOutputCol()], outputCol=[features+'_vec'])
#    stages += [indexers, encoder]

AttributeError: 'StringIndexerModel' object has no attribute 'getOutputCol'

In [35]:
#n_cols = len(x.columns)
#from keras.utils import to_categorical
import numpy as np
import pandas as pd
indexers = [
    StringIndexer(inputCol=column, outputCol=column+"_index", handleInvalid="keep").fit(data) 
    for column in data.columns
]

indexed = indexers.transform(data)
encoders = OneHotEncoderEstimator(inputCols=['magnitude_index'], outputCols=['magnitude_vec']) 
encoded = encoders.transform(indexed)
#pipeline = Pipeline(stages=[indexers, encoders])
#data_r = pipeline.fit(data)
#transformed = data_r.transform(data)
#dataindex = data_r
#for col in data.columns:
#    dataindex = dataindex.drop(col)
#dataindex.show()
#x1 = dataindex.drop('magnitude_index')
#y1 = dataindex[['magnitude_index']]
#dataindex.columns
#x1train, x1test = x1.randomSplit([0.66, 0.34])
#y1train, y1test = y1.randomSplit([0.66, 0.34])
#x1train.columns
#numpycol = ['sourcePort_index',
# 'destinationPort_index',
# 'protocolName_index',
# 'IPgeo_index',
# 'deviceId_index',
# 'event_category_index',
# 'categoryDescription_index',
# 'eventDescription_index',
# 'relevance_index',
# 'credibility_index',
# 'severity_index',
# 'eventCount_index']

#x1train = np.array()
#x1train.show()
#y1train.show()



#pipeline = Pipeline(stages=encoders)
#datapipe = pipeline.fit(data_r).transform(data_r)
#datapipe = data_r
#for col in data.columns:
#    datapipe = datapipe.drop(col)
#datapipe.show()

#for col in data_r.columns:
#    data_x = data_x.drop(col)
#encoders = [
#    OneHotEncoder(inputCol=column, outputCol=column+"_vec") 
#    for column in y1.columns
#]
#pipeline = Pipeline(stages=encoders)
#data_y = pipeline.fit(y1).transform(y1)
#for col in data_r.columns:
#    data_y = data_y.drop(col)
#data_x.printSchema()
#data_y.printSchema()
#data_xtrain, data_xtest = data_x.randomSplit([0.66, 0.34])
#data_ytrain, data_ytest = data_y.randomSplit([0.66, 0.34])
#print(type(data_xtrain))

#print(data_xtrain)

#indexer = StringIndexer(inputCol = 'sourcePort', outputCol = 'scplbl')
#encoder = OneHotEncoder(inputCol = 'scplbl', outputCol='scplblvec')
#pipeline = Pipeline(stages=[indexer, encoder])
#model=pipeline.fit(data)
#transformed = model.transform(data)
#transformed.show()
#data.show()

#def string_to_float(x):
#    return float(x)
#udfstring_to_float = udf(string_to_float, StringType())
#data.withColumn('sourcePort', udfstring_to_float("numberfloat"))
#data.printSchema()
#indexer = StringIndexer(inputCol = 'sourcePort', outputCol = 'scplbl')
#encoder = OneHoteEncoder(inputCol = )

AttributeError: 'list' object has no attribute 'transform'

In [18]:
#from keras.utils import to_categorical
dataindex.columns
dataindexnp = np.array(dataindex.select('sourcePort_index',
 'destinationPort_index',
 'protocolName_index',
 'IPgeo_index',
 'deviceId_index',
 'event_category_index',
 'categoryDescription_index',
 'eventDescription_index',
 'relevance_index',
 'credibility_index',
 'severity_index',
 'magnitude_index',
 'eventCount_index').collect())
ytrainnp = to_categorical(dataindex.magnitude_index)

ValueError: setting an array element with a sequence.

In [14]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import RMSprop, SGD

model = Sequential()

#get number of columns in training data
notnneded, n_cols_2 =xtrainnp.shape 

#add layers to model
model.add(Dense(250, activation='relu', input_shape=(n_cols_2,)))
model.add(Dropout(0.5))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(xtrainnp, ytrainnp, epochs=10, batch_size=64)
score = model.evaluate(xtestnp, ytestnp, batch_size=128)

ValueError: Error when checking target: expected dense_12 to have shape (2,) but got array with shape (1,)

In [ ]:
#from pyspark.ml import Pipeline
#from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, Normalizer
#from pyspark.sql.functions import udf
#from pyspark.sql.types import *
# All the code below to the double space works for single column
#data = df_data_1.select('sourcePort', 'IPgeo', 'protocolName')
#data = data.fillna('Unknown')
#data.show()
#data = data.withColumn('sourcePort', data['sourcePort'].cast(IntegerType()))
#This code works
#indexer = StringIndexer(inputCol = 'IPgeo', outputCol = 'ipglbl')
#indexed_df = indexer.fit(data).transform(data)
#encoder = OneHotEncoder(inputCol = 'ipglbl', outputCol='ipglblvec')
#encoded_df = encoder.transform(indexed_df)
#encoded_df.show()

#indexer = StringIndexer(inputCol = 'sourcePort', outputCol = 'scplbl')
#encoder = OneHotEncoder(inputCol = 'scplbl', outputCol='scplblvec')
#pipeline = Pipeline(stages=[indexer, encoder])
#model=pipeline.fit(data)
#transformed = model.transform(data)
#transformed.show()
#data.show()

#def string_to_float(x):
#    return float(x)
#udfstring_to_float = udf(string_to_float, StringType())
#data.withColumn('sourcePort', udfstring_to_float("numberfloat"))
#data.printSchema()
#indexer = StringIndexer(inputCol = 'sourcePort', outputCol = 'scplbl')
#encoder = OneHoteEncoder(inputCol = )